In [33]:
import sys
import random
from enum import Enum
import os
import subprocess

In [34]:
class HeapType(Enum):
    MIN = "min"
    MAX = "max"

class HeapifyStrategy(Enum):
    UP = "up"
    DOWN = "down"

class CustomHeap:
    def __init__(self, heap_type=HeapType.MIN, filename_prefix="heap_graph"):
        self.heap = [None]  # 1-based indexing
        self.heap_type = heap_type  # Enum: MIN or MAX
        self.sorted_output = []  # Stores deleted elements in sorted order
        self.filename_prefix = filename_prefix
        self.output_folder = "output"
        os.makedirs(self.output_folder, exist_ok=True)
        self.output_folder_pdf = "output_pdf"
        os.makedirs(self.output_folder_pdf, exist_ok=True)
        self.file_counter = 0

    # Time Complexity: Theta(1)
    # Space Complexity: Theta(1)
    def compare(self, a, b, compares):
        compares += 1
        return (a < b if self.heap_type == HeapType.MIN else a > b), compares

    # Time Complexity: Theta(1)
    # Space Complexity: Theta(1)
    def swap(self, i, j, swaps):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        swaps += 1
        return swaps
    
    # Time Complexity: O(logn)
    # Space Complexity: Theta(1)
    def heapify_up(self, index):
        compares, swaps = 0, 0
        while index > 1:
            parent = index // 2
            should_swap, compares = self.compare(self.heap[index], self.heap[parent], compares)
            if should_swap:
                swaps = self.swap(index, parent, swaps)
                index = parent
            else:
                break
        return {"compares": compares, "swaps": swaps}
    
    # Time Complexity: O(logn)
    # Space Complexity: Theta(1)
    def heapify_down(self, index):
        compares, swaps = 0, 0
        while 2 * index < len(self.heap):
            left = 2 * index
            right = left + 1
            smallest_or_largest = left
            
            if right < len(self.heap):
                is_right_better, compares = self.compare(self.heap[right], self.heap[left], compares)
                if is_right_better:
                    smallest_or_largest = right
            
            should_swap, compares = self.compare(self.heap[smallest_or_largest], self.heap[index], compares)
            if should_swap:
                swaps = self.swap(index, smallest_or_largest, swaps)
                index = smallest_or_largest
            else:
                break                
        return {"compares": compares, "swaps": swaps}
    
    # Time Complexity: DOWN: O(n), UP: O(nlogn)
    # Space Complexity: Theta(1)
    def build_heap(self, arr, strategy=HeapifyStrategy.DOWN):
        # Time Complexity: O(nlogn)
        # Space Complexity: Theta(1)
        if strategy == HeapifyStrategy.UP:
            self.heap = [None]
            self.heap.append(arr[0])
            total_compares, total_swaps = 0, 0
            self.save_graphviz(total_compares, total_swaps)
            for i in range(2, len(arr) + 1):  
                self.heap.append(arr[i-1])
                result = self.heapify_up(i)
                total_compares += result["compares"]
                total_swaps += result["swaps"]
                self.save_graphviz(total_compares, total_swaps)
        # Time Complexity: O(n)
        # Space Complexity: Theta(1)
        else:  
            self.heap = [None] + arr
            total_compares, total_swaps = 0, 0
            self.save_graphviz(total_compares, total_swaps)
            for i in range((len(self.heap) - 1) // 2, 0, -1):
                result = self.heapify_down(i)
                total_compares += result["compares"]
                total_swaps += result["swaps"]
                self.save_graphviz(total_compares, total_swaps)

        return {"compares": total_compares, "swaps": total_swaps}
    
    # Time Complexity: O(n)
    # Space Complexity: Theta(1)
    def get_heap(self):
        return self.heap[1:]

    # Time Complexity: O(logn)
    # Space Complexity: Theta(1)
    def delete_root(self):
        if len(self.heap) <= 1:
            return None, {"compares": 0, "swaps": 0}

        root_value = self.heap[1]
        self.heap[1] = self.heap[-1]  
        self.heap.pop()  
        self.sorted_output.append(root_value)
        
        if len(self.heap) > 1:
            result = self.heapify_down(1)
        else:
            result = {"compares": 0, "swaps": 0}

        return root_value, result # returns deleted value and heapify result
    
    # Time Complexity: O(nlogn)
    # Space Complexity: Theta(1)
    def delete_heap(self):
        self.sorted_output = []
        total_compares, total_swaps = 0, 0
        self.save_graphviz(total_compares, total_swaps)
        
        while len(self.heap) > 1:
            _, result = self.delete_root()
            total_compares += result["compares"]
            total_swaps += result["swaps"]
            self.save_graphviz(total_compares, total_swaps, delete=True)

        return self.sorted_output, {"compares": total_compares, "swaps": total_swaps} # returns sorted output and total compares and swaps
    
    # Time Complexity: O(n)
    # Space Complexity: Theta(1)
    def save_graphviz(self, compares, swaps, delete=False):
        # self.file_counter += 1
        # filename = os.path.join(self.output_folder, f"{self.filename_prefix}_{self.file_counter}.dot")
        # dot_representation = self.graphviz(compares, swaps)
        # with open(filename, "w") as f:
        #     f.write(dot_representation)
        self.file_counter += 1
        del_prefix = "delete_" if delete else ""
        dot_filename = os.path.join(self.output_folder, f"{del_prefix}{self.filename_prefix}_{self.file_counter}.dot")
        pdf_filename = os.path.join(self.output_folder_pdf, f"{del_prefix}{self.filename_prefix}_{self.file_counter}.pdf")
        
        dot_representation = self.graphviz(compares, swaps)
        with open(dot_filename, "w") as f:
            f.write(dot_representation)
        # generate PDF from dot files
        try:
            subprocess.run(["dot", "-Tpdf", dot_filename, "-o", pdf_filename], check=True)
        except FileNotFoundError:
            print("Graphviz 'dot' command not found. Please install Graphviz to generate PDFs.")
        except subprocess.CalledProcessError:
            print(f"Error generating PDF for {dot_filename}.")
    
    # Time Complexity: O(n)
    # Space Complexity: Theta(1)
    def graphviz(self, compares, swaps):
        heap = self.get_heap()
        n = len(heap)
    
        digraph_str = ["digraph g {"]
        digraph_str.append("node [shape=record];")       
    
        value_repr = "|".join(f"{{{i+1}|{heap[i]}}}" for i in range(n)) 
        digraph_str.append(f'a [label="{value_repr}"];')

        # Print the sorted output
        output_repr = "|".join(f"{val}" for val in self.sorted_output) if self.sorted_output else "None"
        digraph_str.append(f'c [label="Sorted Output: {output_repr}"];')

        # Print the number of comparisons and swaps
        digraph_str.append(f'b [label="Comparisons: {compares} | Swaps: {swaps}"];')
    
        for i in range(n):
            digraph_str.append(f'{i} [label="{heap[i]}", xlabel="{i+1}"];')
    
        for i in range(n):
            left = 2 * i + 1
            right = left + 1
            # to differentiate between left and right children
            if left < n:
                digraph_str.append(f'edge [color=blue] {i} -> {left};')
            if right < n:
                digraph_str.append(f'edge [color=red] {i} -> {right};')
    
        digraph_str.append("}")
        return "\n".join(digraph_str)


In [35]:
def test_heap():
    n = 10
    a_rand = random.sample(range(-100, 101), 10)
    a_asc = [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
    a_desc = sorted(a_asc, reverse=True)
    
    test_cases = [ ("Ascending", a_asc), ("Descending", a_desc), ("Random", a_rand)]
    heap_types = [HeapType.MIN, HeapType.MAX]
    strategies = [HeapifyStrategy.UP, HeapifyStrategy.DOWN]
    
    for name, arr in test_cases:
        for heap_type in heap_types:
            for strategy in strategies:
                print(f"\nTesting {name} array with {heap_type.value} heap using {strategy.value} strategy")
                heap = CustomHeap(heap_type, filename_prefix=f"{name.lower()}_{heap_type.value}_{strategy.value}")
                result = heap.build_heap(arr, strategy)
                print(f"Heap: {heap.get_heap()}")
                print(f"Comparisons: {result['compares']}, Swaps: {result['swaps']}")
                if (strategy == HeapifyStrategy.DOWN):
                    sorted_output, result = heap.delete_heap()
                    print(f"Sorted Output: {sorted_output}")
                    print(f"Comparisons: {result['compares']}, Swaps: {result['swaps']}")
                

if __name__ == "__main__":
    test_heap()



Testing Ascending array with min heap using up strategy
Heap: [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Comparisons: 9, Swaps: 0

Testing Ascending array with min heap using down strategy
Heap: [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Comparisons: 9, Swaps: 0
Sorted Output: [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Comparisons: 26, Swaps: 12

Testing Ascending array with max heap using up strategy
Heap: [20, 19, 16, 17, 18, 12, 15, 11, 14, 13]
Comparisons: 19, Swaps: 19

Testing Ascending array with max heap using down strategy
Heap: [20, 19, 17, 18, 15, 16, 13, 11, 14, 12]
Comparisons: 14, Swaps: 8
Sorted Output: [20, 19, 18, 17, 16, 15, 14, 13, 12, 11]
Comparisons: 27, Swaps: 13

Testing Descending array with min heap using up strategy
Heap: [11, 12, 15, 14, 13, 19, 16, 20, 17, 18]
Comparisons: 19, Swaps: 19

Testing Descending array with min heap using down strategy
Heap: [11, 12, 14, 13, 16, 15, 18, 20, 17, 19]
Comparisons: 14, Swaps: 8
Sorted Output: [11, 12, 13, 14, 15, 16, 17, 18